## Region-Based Segmentation

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow

# access google drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# prompt: load zip file from drive

import zipfile

# Replace 'path/to/your/zipfile.zip' with the actual path to your zip file in Google Drive
zip_path = '/content/drive/MyDrive/Colab Notebooks/MSFD.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/') # Extracts to the current working directory (/content/)


### Otsu thresholding

In [ ]:
import cv2
import numpy as np
import os
import random

def calculate_IoU(original_image, thresholded_image):
    # Compute IoU
    # Reshape to the shape fo the thresholded_image
    original_image = cv2.resize(original_image, (thresholded_image.shape[1], thresholded_image.shape[0]))
    intersection = np.logical_and(original_image, thresholded_image)
    union = np.logical_or(original_image, thresholded_image)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

def calculate_Dice(original_image, thresholded_image):
    # calculate dice score
    # Reshape to the shape fo the thresholded_image
    original_image = cv2.resize(original_image, (thresholded_image.shape[1], thresholded_image.shape[0]))
    intersection = np.logical_and(original_image, thresholded_image)
    dice_score = 2.0 * np.sum(intersection) / (np.sum(original_image) + np.sum(thresholded_image))
    return dice_score

train_folder = '/content/MSFD/1/face_crop'
test_folder = '/content/MSFD/1/face_crop_segmentation'

train_images = os.listdir(train_folder)
test_images = os.listdir(test_folder)

# Create sets for quick lookup
test_image_set = set(test_images)

total_iou = 0
total_dice = 0
image_count = 0
unmatched_count = 0

for train_filename in train_images:
    if train_filename in test_image_set:
        image_path = os.path.join(train_folder, train_filename)
        gt_path = os.path.join(test_folder, train_filename)

        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        gt = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

        if image is not None and gt is not None:
            # Apply Otsu's thresholding
            _, thresholded_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            iou = calculate_IoU(gt, thresholded_image)
            total_iou += iou
            dice = calculate_Dice(gt, thresholded_image)
            total_dice += dice
            image_count += 1
    else:
        unmatched_count += 1

if image_count > 0:
    average_iou = total_iou / image_count
    average_dice = total_dice / image_count
    print("IoU for Otsu thresholding:", average_iou)
    print("Dice for Otsu thresholding:", average_dice)
    print("Unmatched images:", unmatched_count)
else:
    print("No matching images found in the specified folders.")


### Trainable

In [ ]:
import cv2
import numpy as np
import glob

def mse(mask1, mask2):
    """Compute Mean Squared Error"""
    return np.mean((mask1.astype("float") - mask2.astype("float")) ** 2)

def optimize_thresholds(image_paths, gt_paths):
    best_mse = float('inf')
    best_thresholds = None

    # Try different HSV threshold values
    for h_low in range(0, 180, 10):
        for h_high in range(h_low+10, 180, 10):
            for s_low in range(50, 255, 20):
                for s_high in range(s_low+20, 255, 20):
                    for v_low in range(50, 255, 20):
                        for v_high in range(v_low+20, 255, 20):

                            total_mse = 0
                            count = 0

                            for img_path, gt_path in zip(image_paths, gt_paths):
                                img = cv2.imread(img_path)
                                gt = cv2.imread(gt_path, 0)  # Load GT mask in grayscale
                                gt = cv2.threshold(gt, 127, 255, cv2.THRESH_BINARY)[1]  # Binarize

                                # Convert to HSV and apply threshold
                                hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
                                lower = np.array([h_low, s_low, v_low])
                                upper = np.array([h_high, s_high, v_high])
                                mask = cv2.inRange(hsv, lower, upper)

                                # Compute MSE
                                total_mse += mse(mask, gt)
                                count += 1

                            avg_mse = total_mse / count

                            if avg_mse < best_mse:
                                best_mse = avg_mse
                                best_thresholds = (h_low, h_high, s_low, s_high, v_low, v_high)

    return best_thresholds, best_mse

# Load dataset
path_to_images = '/content/MSFD/1/face_crop'
path_to_gt = '/content/MSFD/1/face_crop_segmentation'
image_paths = sorted(glob.glob(path_to_images + "/*.jpg"))  # Modify with actual path
gt_paths = sorted(glob.glob(path_to_gt + "/*.jpg"))  # Modify with actual path

best_thresholds, best_mse = optimize_thresholds(image_paths, gt_paths)

print("Best Thresholds:", best_thresholds)
print("Lowest MSE:", best_mse)


## Classification

In [ ]:
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

### CNN Classifier

In [ ]:
input_shape = (128, 128, 3)

#### Model

In [ ]:
from keras import layers, models, Sequential
from keras.regularizers import l2

model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

#### Load a Model

In [ ]:
# prompt: load a model
from keras import models

model = models.load_model("/content/mask_detection_model_small.h5")

model.summary()

#### Training

In [ ]:
from keras.optimizers import Adam, SGD
from keras.losses import BinaryCrossentropy

optimizers = {
    "adam": Adam(learning_rate=0.001),
    "sgd": SGD(learning_rate=0.001, momentum=0.9)
}

losses = {
    "bce": BinaryCrossentropy(label_smoothing=0.10),
    "scce": "sparse_categorical_crossentropy"
}
model.compile(optimizer=optimizers["adam"], loss=losses["bce"], metrics=['accuracy'])



In [ ]:
# preprocessing:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define ImageDataGenerator for real-time augmentation and memory efficiency
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalize pixel values

batch_size = 32
# Load train and validation sets using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    'Face-Mask-Detection/dataset/',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    subset='training'  # Training set
)

val_generator = train_datagen.flow_from_directory(
    'Face-Mask-Detection/dataset/',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'  # Validation set
)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Get class labels from generator
labels = train_generator.classes  # Get all training labels

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights_dict = dict(enumerate(class_weights))

print("Class Weights:", class_weights)

val_labels = val_generator.classes  # Get all validation labels
val_class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(val_labels), y=val_labels)
val_class_weights_dict = dict(enumerate(val_class_weights))

print("Validation Class Weights:", val_class_weights)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Keep patience at 10
    min_delta=0.001,  # Ignore very small improvements
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=20,
    class_weight=class_weights_dict,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve

# Get prediction probabilities
y_pred_probs = model.predict(val_generator)
y_true = val_generator.classes

# Handle cases where model outputs two probabilities (for binary classification)
if y_pred_probs.shape[1] == 2:
    y_pred_probs = y_pred_probs[:, 1]  # Use probability of the positive class

# Compute precision-recall curve
precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred_probs)

# Compute F1 scores
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)  # Avoid division by zero

# Find best threshold (maximize F1-score)
best_threshold = thresholds[np.argmax(f1_scores)]

print(f"Best Threshold: {best_threshold:.4f}")

# Apply threshold for classification
y_pred_new = (y_pred_probs > best_threshold).astype(int)


In [ ]:
import matplotlib.pyplot as plt

plt.hist(y_pred_probs, bins=50, edgecolor="black")
plt.xlabel("Predicted Probability")
plt.ylabel("Frequency")
plt.title("Distribution of Predicted Probabilities")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Get predictions (probabilities)
y_pred = model.predict(val_generator)

# Handle cases where model outputs two probabilities (for binary classification)
if y_pred.shape[1] == 2:
    y_pred = y_pred[:, 1]  # Take probability of the positive class

# Convert probabilities to class labels using a default threshold of 0.5
y_pred_classes = (y_pred > 0.5).astype(int)

# Get true labels
y_true = val_generator.classes

# Print Classification Report
print(classification_report(y_true, y_pred_classes, target_names=['No Mask', 'Mask']))


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.show()

In [ ]:
# save the model
model.save('mask_detection_model_small.h5')

#### Evaluating on entire dataset

In [ ]:
# load the model
from tensorflow.keras.models import load_model
# model = load_model('mask_detection_model_first.h5')

# evaluate on entire dataset
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'Face-Mask-Detection/dataset/',
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

### Generated

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.manifold import TSNE

# Data Paths
data_dir = "/content/Face-Mask-Detection/dataset"  # UPDATE this to your dataset folder

# Image Data Generator with Augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% train, 20% val
)

# Load Data
img_size = (128, 128)
batch_size = 32

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Improved CNN Model
def build_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(128, 128, 3)),
        BatchNormalization(),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Flatten(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,  # Increased for better feature learning
    callbacks=[early_stopping, lr_reduction]
)

# Evaluate Model
y_pred_probs = model.predict(val_generator)
y_true = val_generator.classes
y_pred_classes = (y_pred_probs > 0.5).astype(int)

print(classification_report(y_true, y_pred_classes, target_names=['No Mask', 'Mask']))

# Precision-Recall Curve
precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred_probs)
plt.plot(recalls, precisions, label="Precision-Recall Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.show()
